# 1.1. Описание проекта

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.\
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

# 1.2. Инструкция по выполнению

- Загрузите и изучите данные.
- Ответьте на вопрос и обоснуйте решение.
- Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
  - a. Изменится. Приведите примеры матриц.
  - b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
- Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.
- Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

# 1.3. Описание данных

Набор данных находится в файле /datasets/insurance.csv. \
Признаки: 
- Пол
- Возраст
- Зарплата застрахованного
- Количество членов его семьи.

Целевой признак: 
- Количество страховых выплат клиенту за последние 5 лет.

# 2. Подготовка данных

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
df = pd.read_csv('insurance.csv')
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


# 3. Работа с данными (умножение матриц)

Алгоритм

Создаётся квадратная матрица размером равным числу параметров (можно числа наблюдений, но придётся транспонировать матрицу признаков) - encription_key\
Находится инвертированная к ней матрица decription_key\
Признаки кодируются с помощью матрицы encription_key (путём умножения матрицы признаков на encription_key). \
Преобразованные признаки и целевой параметр являются защищёнными исходными данными.\
Преобразованные признаки можно преобразовать к исходным, зная инвертированную матрицу (путём умножения на неё).

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X = df.drop(['Страховые выплаты'], axis=1)
y = df['Страховые выплаты'] 

print('Признаки до кодирования: ')
print(X.head())
print()

Признаки до кодирования: 
   Пол  Возраст  Зарплата  Члены семьи
0    1     41.0   49600.0            1
1    0     46.0   38000.0            1
2    0     29.0   21000.0            0
3    0     21.0   41700.0            2
4    1     28.0   26100.0            0



In [11]:
encription_key = np.random.normal(size=(X.T.shape[0],X.T.shape[0]) ) 
decription_key = np.linalg.inv(encription_key)
Coded = (X @ encription_key) 

print('Признаки после кодирования: ') 
print(Coded.head()) 
print()

Признаки после кодирования: 
              0             1             2             3
0  73253.532407  29099.883098 -58432.255715  32157.014846
1  56106.861959  22291.844311 -44763.808991  24657.998233
2  31001.599001  12318.633451 -24737.528067  13631.757696
3  61602.594295  24467.884881 -49127.895102  27015.872919
4  38538.911199  15311.219230 -30746.427000  16930.593640



In [12]:
print('Признаки после декодирования: ')
Decoded = (Coded @ decription_key).astype('int') 
print(Decoded.head())

Признаки после декодирования: 
   0   1      2  3
0  1  41  49599  1
1  0  46  38000  0
2  0  29  21000  0
3  0  21  41699  2
4  1  28  26099  0


In [13]:
model_1 = LinearRegression()
model_1.fit(X, y)
predictions_1 = model_1.predict(X)
print('Нативные параметры, R2=',r2_score(y, predictions_1))

model_2 = LinearRegression()
model_2.fit(Coded, y)
predictions_2 = model_2.predict(Coded)
print('Закодированные параметры, R2=',r2_score(y, predictions_2))
    

Нативные параметры, R2= 0.4249455028666801
Закодированные параметры, R2= 0.424945502866682
